# HanLP MTL dùng ELECTRA_BASE
Chạy công cụ phân tích ngữ pháp chương tiết trên Chivi.

In [1]:
# @title ## Bước 1: Cài đặt thư viện
# @markdown Cần chạy trước khi chạy các bước khác.
%pip install hanlp requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
#@title ## Bước 2: Tải xuống dữ liệu
# @markdown Tải xuống thư viện liên quan, bước này có thể sẽ hơi lâu, nếu hệ thống bận.

import os, re, requests, getpass

URL = "https://chivi.app/_wn"
CLIENT = requests.Session()

os.environ["HANLP_HOME"] = ".hanlp" # đổi địa chỉ lưu models
import hanlp

# tải xuống dữ liệu 
MTL_TYPE = hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH
MTL_TASK = hanlp.load(MTL_TYPE)

# lấy thông tin chương cuối cùng
def get_max_ch_no(wn_id, sname):
  url = f"{URL}/seeds/{wn_id}/{sname}/brief"
  res = CLIENT.get(url)
  return res.json()["chap_count"]

# đảm bảo dữ liệu cây ngữ pháp chỉ có một dòng
def render_con_data(con_data):
  output = ''

  for con_line in con_data:
    output += re.sub('\\n(\\s+)', ' ', str(con_line))
    output += '\n'

  return output

# phản hồi tới người dùng, bỏ qua phần đệm (chỉ chứa tên chương) mỗi chương
def print_msg(message, p_idx):
  if p_idx > 0:
    print(message % p_idx)

def invoke_mtl_task(lines):
  try: 
    # thử chạy phân tích tất cả các dòng
    return MTL_TASK(lines)
  except:
    # nếu tràn bộ nhớ, thử phân tích từng dòng một
    mtl_data = MTL_TASK([lines[0]])

    for line in lines[1:]:
      mtl_line = MTL_TASK(line)

      for key in mtl_data:
        mtl_data[key].append(mtl_line[key])

    return mtl_data

# phân tích ngữ pháp từng phần mỗi chương
def analyze_text(ptext, wn_id, sname, ch_no, cksum, p_idx):
  mtl_data = invoke_mtl_task(ptext.split('\n'))
  print_msg("  - Đã phân tích xong phần %s, đang tải lên...", p_idx)

  json = {
    'wn_id': wn_id, 'sname': sname,
    'ch_no': ch_no, 'cksum': cksum,
    'p_idx': p_idx, '_algo': 'hmeb',
    'con_text': render_con_data(mtl_data["con"]),
    'mtl_json': mtl_data.to_json(),
  }

  url = "https://chivi.app/_wn/anlzs/chaps"
  res = CLIENT.post(url, json = json)

  if res.status_code > 300:
    print(f"  Đăng tải thất bại: {res.text}")

def analyze_book(wn_id, sname, chmin = 1, chmax = 1, force = False):
  for ch_no in range(chmin, chmax + 1):
    print(f"\n- Tải xuống text gốc chương [{ch_no}] nguồn [{sname}] truyện [{wn_id}]:")

    url = f"{URL}/texts/{wn_id}/{sname}/{ch_no}/parts"
    res = CLIENT.get(url).json()

    cksum = res["cksum"]
    parts = res["parts"]
    _hmeb = res["_hmeb"]

    print(f"  Đã lấy xuống text gốc, chương chia {len(parts) - 1} phần, checksum: [{cksum}].")
    print()

    for p_idx, ptext in enumerate(parts):
      if _hmeb[p_idx] == True and not force:
        print_msg("  - Phần %s đã có trên hệ thống, bỏ qua.", p_idx)
        continue

      try:
        analyze_text(ptext, wn_id, sname, ch_no, cksum, p_idx)
      except Exception as inst:
        print(inst)

 27% 448.0 MiB   1.1 MiB/s ETA:  5 m  1 s [=========                           ]

Download failed due to KeyboardInterrupt().
Please re-try or download it to .hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip by yourself with:

	wget https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip -O .hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip

See also https://hanlp.hankcs.com/docs/install.html#install-models for instructions.


KeyboardInterrupt: 

In [ ]:
#@title ## Bước 3: Đăng nhập vào hệ thống
# @markdown Bạn hãy cung cấp địa chỉ email + mật khẩu tài khoản Chivi để hệ thống định danh

# đăng nhập vào hệ thống
def do_login():
  print("\nThông tin đăng nhập Chivi:")

  # @markdown Địa chỉ hòm thư:
  email = '' # @param {type: "string"}
  # @markdown Mật khẩu:
  upass = '' # @param {type: "string", hidden: true}

  if email == '':
    print("Hòm thư:", end=' ')
    email = input().strip()

  if upass == '':  
    upass = getpass.getpass("Mật khẩu: ").strip()

  json = {'email': email, 'upass': upass}
  res = CLIENT.post("https://chivi.app/_db/_user/log-in", json=json)

  if res.status_code < 300:
    user = res.json()["uname"]
    print(f"Xin chào {user}!")
    print()
  else:
    print("Đăng nhập thất bại, mời thử lại!")
    do_login()
    
do_login()

In [ ]:
#@title ## Bước 4: Phân tích ngữ pháp

# @markdown *ID bộ truyện:* Con số đầu tiên trên địa chỉ bộ truyện. Ví dụ truyện có địa chỉ là https://chivi.app/wn/4-quy-bi-chi-chu/ thì ID bộ truyện là 4
# @markdown *Nguồn chương:* Tên danh sách chương, nguồn chính thức tên là ~chivi, nguồn tạm thời tên là ~draft, các nguồn cá nhân bắt đầu bằng @ tiếp đó là tên đăng nhập.

# @markdown ID bộ truyện:
wn_id = 0 # @param {type: "integer"}

# @markdown Nguồn chương:
sname = '~chivi' # @param {type: "string"}

# @markdown Chương bắt đầu
chmin = 1 # @param {type: "integer"}

# @markdown Chương kết thúc
chmax = 0 # @param {type: "integer"}

# @markdown Phân tích lại dù có trên hệ thống
force = False # @param {type: "boolean"}

if wn_id == 0:
  print('Nhập ID bộ truyện:', end=' ')
  wn_id = int(input().strip())

if sname == '':
  print("Nhập nguồn chương:", end=' ')
  sname = input().strip()

if sname == '':
  sname = "~chivi" 

print("...Kiểm tra dữ liệu...", end=' ')

url = f"{URL}/seeds/{wn_id}/{sname}/brief"
res = requests.get(url).json()

total = res["chap_count"]
print(f"hoàn thành, nguồn truyện có tổng cộng {total} chương")

if chmax < chmin:
  chmax = total

if chmin < 1:
  chmin = 1

print(f"Bạn sẽ chạy chương trình phân tích dữ liệu các chương từ {chmin} tới {chmax} của bộ truyện {wn_id}")

analyze_book(wn_id, sname, chmin, chmax, force)